In [1]:
setwd(paste0(Sys.getenv('ROOT'), '/R'))
getwd()

[1] "/mnt/hdd/develop/kaggle/house-prices/R/R"

In [9]:
source('main.R')

data <- kaggle.house$loadData()

df.combined <- kaggle.house$getCombinedDataset(data$train, data$test) %>% (kaggle.house$na$fixAll)

df.training <- 
    df.combined %>% 
    filter(dataSource == "train") %>% 
    mutate(sale_price_log = log(SalePrice)) %>%
    select(-dataSource, -SalePrice)

In [10]:
source('main.R')
trainset <- kaggle.house$trans$doItAll(df.training)
# Should be only numeric columns
stopifnot(0 == trainset %>% purrr::map(function (col) { !is.numeric(col) }) %>% unlist %>% sum)

In [20]:
trainset %>% 
    gather(name, value, -sale_price_log) %>%
    mutate(value.log = log(value + 1)) %>%
    group_by(name) %>% nest %>%
    mutate(
        model =  map(data, ~lm(sale_price_log ~ value, .)),
        glance = map(model, broom::glance),
        r2 = map_dbl(glance, 'r.squared'),
        
        model.log =  map(data, ~lm(sale_price_log ~ value.log, .)),
        glance.log = map(model.log, broom::glance),
        r2.log = map_dbl(glance.log, 'r.squared'),
        
        best.r2 = pmax(r2, r2.log)
    ) %>%
    select(name, r2, r2.log, best.r2) %>%
    arrange(desc(best.r2)) %>%
    filter(best.r2 > 0.1)
    

name,r2,r2.log,best.r2
OverallQual,0.66779037,0.64331216,0.6677904
Neighborhood.new,0.56401649,0.56407646,0.5640765
GrLivArea,0.49129817,0.53327044,0.5332704
GarageCars,0.46325013,0.41160074,0.4632501
ExterQual.new,0.46037836,0.46068150,0.4606815
BsmtQual.new,0.45120773,0.45163675,0.4516368
KitchenQual.new,0.44377503,0.44446922,0.4444692
GarageArea,0.42365461,0.20693294,0.4236546
GarageFinish.new,0.38079048,0.38073806,0.3807905
TotalBsmtSF,0.37470800,0.13900807,0.3747080
